In [4]:
# mount the google drive to access its content.
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)


Mounted at /content/gdrive


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
!pip install pyspark

In [7]:
# import libraries
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, desc, avg, mean

In [8]:
# read excel file
fp = "/content/gdrive/My Drive/Colab Notebooks/Tendo/Exercise Data.xlsx"
sheet_names = ['encounter_e1', 'lab_e1', 'medications_e1', 'patient_e1']

encounter_e1 = pd.read_excel(fp, sheet_name=sheet_names[0])
lab_e1 = pd.read_excel(fp, sheet_name=sheet_names[1])
medications_e1 = pd.read_excel(fp, sheet_name=sheet_names[2])
patient_e1 = pd.read_excel(fp, sheet_name=sheet_names[3])

In [ ]:
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF_encounter_e1=spark.createDataFrame(encounter_e1)
sparkDF_encounter_e1.show()
sparkDF_lab_e1=spark.createDataFrame(lab_e1)
sparkDF_lab_e1.show()
sparkDF_medications_e1=spark.createDataFrame(medications_e1)
sparkDF_medications_e1.show()
sparkDF_patient_e1=spark.createDataFrame(patient_e1)
sparkDF_patient_e1.show()

In [10]:
df = sparkDF_patient_e1.join(sparkDF_encounter_e1,['patientid']) \
                  .join(sparkDF_medications_e1,['patientid','encounterid']).groupBy(sparkDF_patient_e1.patientid,'Sex','Age','primary_care_provider','medication_simple_generic_name','dose_unit','admit_diagnosis') \
                  .agg(avg('minimum_dose').alias('avg_minimum_dose')).sort(desc(sparkDF_patient_e1.patientid))

In [ ]:
df.show(df.count(), False)

In [12]:
from datetime import datetime
filepath = "/content/gdrive/My Drive/Colab Notebooks/Tendo/"
filename = r"target_1_" + datetime.now().strftime("%Y%m%d") + ".txt"
df.toPandas().to_csv(filepath+filename, header=True, index=None, sep='|', mode='a', encoding = 'utf-8', quotechar='"', line_terminator="")